# Exploring data with Scipp

- explore the HTML view and the attributes stored alongside the data
  - `data.attrs['abc'].value` shows HTML view for scalar attrs with `dtype=DataArray`.
- find and plot monitor
- find and plot other meta data
- plot a spectrum
- plot total counts => `sc.sum()`
- add and remove attr (or coord)
- plot all monitors on same plot
- "correct" TOF by shifting it by $2.3~\mu s$
- crop by using slicing and copying
- move the sample?

In [ ]:
data.plot()

In [ ]:
pos = data.coords['sample-position'].copy()
pos += sc.scalar(value=np.array([0.2,0.1,0.1]), unit=sc.units.m)
pos

In [ ]:
sc.plot.plot({f'monitor{i}':data.attrs[f'monitor{i}'].value for i in [1,2,3,4,5]})

In [ ]:
sc.plot.plot({f'monitor{i}':sc.neutron.convert(data.attrs[f'monitor{i}'].value,'tof','wavelength') for i in [1,2,3,4,5]})

In [ ]:
data.attrs['abc'] = sc.scalar(value='abc')
del data.attrs['abc']
#sc.table(data.attrs['EPICS_PUTLOG'].value)
sc.table(data.attrs['A1VCent'].value)
data

In [ ]:
for name, attr in data.attrs.items():
    if attr.dtype == sc.dtype.DataArray:
        print(name)
        if attr.value.dtype != sc.dtype.string:
            display(attr.value.plot())

# Working with masks

1. Masking a prompt pulse.
   - Create a mask from the `'tof'` coord of `data` to mask the region between X and Y $\mu s$.
   - Plot the result to inspect the mask.
   - Pass a `dict` containing `counts` (computed above as `counts = sc.sum(data, 'tof')`) and the equivalent counts computed *after* masking to `sc.plot.plot`.
     Use this to verify that the promt-pulse mask results in removal of counts.

- mask X range
- mask TOF, verify counts has changed
- mask based on counts (combined with shape)
- create circular mask
- create function to mask ring 
- mask based on scattering angle

Hints:
- `del`
- `<=`, `>`, `sc.less`, `sc.equal`

In [ ]:
import scipp as sc
import numpy as np
#data = sc.neutron.load(filename='/mnt/extra/simon/MantidExternalData/MD5/e5c22cf69fdd0d007c29aa51c6537004')
run_number = 49338
data = sc.neutron.load(filename=f'LARMOR000{run_number}')

In [ ]:
counts = sc.sum(data, 'tof')
data.masks['tof'] = data.coords['tof']['tof',1:] < 10000.0 * sc.units.us
sc.plot.plot({'orig':counts, 'masked':sc.sum(data,'tof')})

In [ ]:
data.plot()

In [ ]:
pos = data.coords['position']
x = sc.geometry.x(pos)
y = sc.geometry.y(pos)
z = sc.geometry.z(pos)
data.coords['x'] = x
data.coords['y'] = y
data.coords['z'] = z
data

In [ ]:
data.masks['x'] = x < -0.1 * sc.units.m
del data.masks['x']
data.masks['circle'] = sc.sqrt(x*x + y*y) < 0.1*sc.units.m
del data.masks['circle']
r = sc.sqrt(x*x + y*y)
data.masks['ring'] = (0.14*sc.units.m < r) & (r < 0.15*sc.units.m)
theta = sc.neutron.scattering_angle(data)
data.masks['theta'] = (0.01*sc.units.rad < theta) & (theta < 0.02*sc.units.rad)
phi = sc.atan2(y,x) * ((180.0 * sc.units.deg) / (np.pi * sc.units.rad))
data.masks['wedge'] = (10.0*sc.units.deg < phi) & (phi < 20.0*sc.units.deg)

In [ ]:
sc.neutron.instrument_view(sc.sum(data,'tof'))

In [ ]:
import scipp as sc
import numpy as np
data = sc.neutron.load(filename='/home/simon/data/TrainingCourseData/EQSANS_6071_event.nxs')

In [ ]:
data

In [ ]:
49152//256

In [ ]:
edges = sc.array(dims=['tof'], unit=sc.units.us, values=np.linspace(-1.0, 17000.0, num=1001))
hist = sc.histogram(data, edges)
ny = 256
nx = 49152 // ny
var = sc.reshape(hist.data, dims=['x','y','tof'],shape=(nx,ny,1000))

In [ ]:
var.plot()

- Jupyter Notebooks are the generic GUI for working with `scipp` (the plan is to eventually have technique-specific or instrument-specific widgets or GUIs, where required).
- `scipp` stores data in a **multi-dimensional array** with **labeled (named) dimensions**.
  This is best imagined as `numpy` arrays, without the need to memorize and keep track of dimension order.
- Each array is combined with a **physical unit** into a **variable**.
- Variables are enhanced by **coordinates**.
  Each coordinate is also a variable.
  A variable with associated coordinates is called **data array**.
- Multiple data arrays with aligned coordinates can be combined into a **dataset**.

In [ ]:
a = np.random.rand(2,4)
a

In [ ]:
var = sc.Variable(dims=['time','location'], values=a, unit=sc.units.K)
var

In [ ]:
var['location',2:4]

In [ ]:
time =     sc.Variable(dims=['time'], unit=sc.units.s, values=[20,30])
location = sc.Variable(dims=['location'], unit=sc.units.m, values=np.arange(4))
array =    sc.DataArray(data=var, coords={'time':time, 'location':location})
array

In [ ]:
array.unit = sc.units.m
b = array.copy()
b.coords['time'] *= 2
array + b

In [ ]:
#from mantid.kernel import config
#folder = '/folder/with/downloaded/files'
#config.appendDataSearchDir(folder)
#config.saveConfig(config.getUserFilename())

In [ ]:
run_number = 49338
sample = sc.neutron.load(filename=f'LARMOR000{run_number}')

In [ ]:
sample

### Basic 1D and 2D plots

Plotting is mostly based on `matplotlib`.
Data structures with named dimensions, units, and coordinates allows for meaningful plots by default:

In [ ]:
sample.plot()

Slicing can be used, e.g., to select and plot a single spectrum:

In [ ]:
sample['spectrum', 59155].plot()

Plotting multiple spectra on the same plot is also possible by passing a Python `dict` to `scipp.plot.plot`:

In [ ]:
from scipp.plot import plot
section = sample['tof', 100:150]
plot({'spec1':section['spectrum',59155],'spec2':section['spectrum',59255]})

### Debugging Detectors with Scipp

The LoKI detectors are tubes containing 7 straws each, and there are multiple layers of tubes.
This makes finding, e.g., broken straws in the instrument view difficult and tedious.
The default 2D representation of data is not adequate in this case:

In [ ]:
sample

With scipp we can reshape our data to match this logical layer and sum, e.g., over time-of-flight and pixels within straws.
This yields:

In [ ]:
import sys
sys.path.append('/home/simon/code/ess-legacy/sans')
from loki import LoKI
loki = LoKI()
spectrum_counts = sc.sum(sample, 'tof') # sum is optional, could also keep TOF
pixel_counts = loki.to_logical_dims(spectrum_counts) # reshape
pixel_counts

We can plot the counts in each straw by summing along the `'pixel'` dimensions:

In [ ]:
straw_counts = sc.sum(pixel_counts, 'pixel')
straw_counts.plot(norm='log')

If we instead plot `pixel_counts` without summing along straws, we obtain a plot with a slider along the third dimension.
A profile plot can be enabled as well:

In [ ]:
pixel_counts.plot(axes={'x':'straw', 'y':'tube'})

In this case we observe 4 straws with 0 counts as well as 4 straws with very low counts.
We can define a mask for these using a small LoKI-specific helper:

In [ ]:
pos = sc.neutron.position(sample)
x = sc.geometry.x(pos)
y = sc.geometry.y(pos)
counts = spectrum_counts.data
sample.masks['electronics-error'] = (sc.abs(x) < 0.2 * sc.units.m) \
                                  & (sc.abs(y) < 0.03 * sc.units.m) \
                                  & (counts == 0.0 * sc.units.counts)
print(f"Masking {sc.sum(sample.masks['electronics-error'], 'spectrum').value} bad pixels due to electronics error.")

In [ ]:
# Note that this needs more tuning and masks too much. Better do this after moving detectors?
sample.masks['beam-stop'] = (sc.abs(x) < 0.03 * sc.units.m) & (y < 0.028 * sc.units.m) & (y > -0.016 * sc.units.m)

In [ ]:
sample.masks['tube-ends'] = (x > 0.36 * sc.units.m) | (x < -0.36 * sc.units.m)

In [ ]:
tof = sample.coords['tof']
sample.masks['prompt-pulse'] = (tof['tof',1:] < 1500.0 * sc.units.us) | \
                               ((tof['tof',:-1] > 17500.0 * sc.units.us) & \
                                (tof['tof',1:] < 19000.0 * sc.units.us))

In [ ]:
(sample.masks['tube-ends'] | sample.masks['prompt-pulse']).plot()

In [ ]:
loki.to_logical_dims(sample).plot(norm='log', vmin=1e0, vmax=1e2)

In [ ]:
pixel_counts = loki.to_logical_dims(sc.sum(sample, 'tof'))
plot(pixel_counts, vmax=1000, axes={'y':'tube', 'x':'pixel'})

## Backup slides

### Straw plot against real X

In [ ]:
from loki import LoKI
loki = LoKI()
from scipp.plot import plot
spectrum_counts = sc.sum(sample, 'tof') # sum is optional, could also keep TOF
spectrum_counts.coords['pixel'] = sc.geometry.x(sample.coords['position'])
pixel_counts = loki.to_logical_dims(spectrum_counts) # reshape
plot(pixel_counts, norm='log', axes={'y':'tube', 'x':'pixel'})

In [ ]:
#filename = 'PG3_4844_event'
#tmp = sc.neutron.load(filename=f'{filename}.nxs').bins.sum()
filename = '/home/simon/data/TrainingCourseData/SXD23767.raw'
tmp = sc.neutron.load(filename=f'{filename}')
tmp = sc.sum(tmp, 'tof')
tmp.coords['theta'] = sc.neutron.scattering_angle(tmp)
tmp.coords['phi'] = sc.neutron.scattering_angle(tmp)
pos = sc.neutron.position(tmp)
x = sc.geometry.x(pos)
y = sc.geometry.y(pos)
tmp.coords['phi'] = sc.atan(y/x) + np.pi * sc.units.rad
theta = sc.Variable(dims=['theta'], unit=sc.units.rad, values=np.linspace(0, np.pi/2, num=100))
phi = sc.Variable(dims=['phi'], unit=sc.units.rad, values=np.linspace(0, 2*np.pi, num=100))
binned = sc.bin(tmp, edges=[theta,phi])
binned.plot(resolution={'x':100,'y':100})

When build-in surface cuts are not flexible enough, `scipp` features such as `groupby` can be used to quickly extract groups of pixels:

In [ ]:
from loki import LoKI
loki = LoKI()
sample.coords['layer'] = loki.layers()
sc.neutron.instrument_view(sc.groupby(sample, 'layer').copy(group=1), norm='log', pixel_size=0.01, bins=1)
#del sample.coords['layer']

Using the same mechanism we can create nearly arbitrary other visualizations of the instrument.
For example, we may want to inspect all pixels with low counts rates, e.g., to find issues with detectors.
In this case mask all pixels with less than 100 counts and can check whether we indeed masked all relevant features:

In [ ]:
counts = sc.sum(sample.data, 'tof')
sample.coords['low-counts'] = counts < 100.0*sc.units.counts
sc.neutron.instrument_view(sc.sum(sc.groupby(sample, 'low-counts').copy(group=1), 'tof'), pixel_size=0.005)
#del sample.coords['low-counts']